In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -r DATA
!unzip /content/drive/MyDrive/drowsyiness.zip -d ./DATA

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Define the path to your original dataset directory
original_data_folder = './DATA'

# Define the path to the new dataset directory where resized images will be saved
resized_data_folder = './Resized_DATA'

# Define the target size for resizing
target_height = 224
target_width = 224

import os
import cv2


# Load the pre-trained Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

 

def crop_faces_and_save(input_folder, output_folder, target_size=(227, 227)):
    # Iterate through each image in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):  # Check if the file is an image
            # Load an image
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)

 

            # Convert the image to grayscale (Haar Cascade works on grayscale images)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

 

            # Detect faces in the grayscale image
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

 

            # Create output folder if it doesn't exist
            os.makedirs(output_folder, exist_ok=True)

 

            # Crop and save each detected face
            for i, (x, y, w, h) in enumerate(faces):
                face = image[y:y+h, x:x+w]

 

                # Resize the face to the target size
                resized_face = cv2.resize(face, target_size)

 

                output_path = os.path.join(output_folder, f'face_{i}_{filename}')
                cv2.imwrite(output_path, resized_face)

 

# Process drowsy images
drowsy_input_folder = './DATA/drowsy'
drowsy_output_folder = './RESIZED/drowsy'
crop_faces_and_save(drowsy_input_folder, drowsy_output_folder)

 

# Process non-drowsy images
non_drowsy_input_folder = './DATA/not-drowsy'
non_drowsy_output_folder = './RESIZED/notdrowsy'
crop_faces_and_save(non_drowsy_input_folder, non_drowsy_output_folder)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to your dataset directory
data_folder = './RESIZED'

# Define data generators for training and validation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # If you want to split data into training and validation sets
)

# Load and preprocess the data
batch_size = 32
img_height = 227  # Adjust as per your image size
img_width = 227   # Adjust as per your image size

train_generator = datagen.flow_from_directory(
    data_folder,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Use 'binary' for two classes: 'Drowsy' and 'Non-Drowsy'
    subset='training',    # Use 'validation' if you have a validation split
    classes=['drowsy', 'notdrowsy']
)

# Create a validation generator
validation_generator = datagen.flow_from_directory(
    data_folder,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',  # Use 'training' if you have a training split
    classes=['drowsy', 'notdrowsy']
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Create a sequential model
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output
model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Dropout for regularization
model.add(Dense(1, activation='sigmoid'))  # Output layer with one neuron for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
epochs = 10  # You can adjust the number of epochs
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)


In [ ]:
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {validation_accuracy:.2f}')

In [ ]:
model.save('drowsiness_detection_model_rnn.h5')